In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import current_timestamp, trim, col, to_date, round

StatementMeta(, 9f6bb14d-2ee8-464f-89c0-fadc41cfda23, 4, Finished, Available, Finished)

In [3]:
# 1️⃣ Read Bronze
df = spark.read.format("delta").table("BronzeLakehouse.dbo.Carts")

# 2️⃣ Clean and select columns
resultant_df = df.select(
    col("Cart_ID"),
    col("Customer_ID"),
    col("Product_ID"),
    trim(col("product_title")).alias("Product_Title"),
    col("price").cast("double").alias("Price"),
    col("quantity").cast("int").alias("Quantity"),
    col("discountPercentage").cast("double").alias("DiscountPercentage"),
    col("total").cast("double").alias("Total"),
    col("discountedTotal").cast("double").alias("DiscountedTotal"),
    col("cart_total").cast("double").alias("Cart_Total"),
    col("cart_discounted_total").cast("double").alias("Cart_Discounted_Total"),
    col("cart_total_products").cast("int").alias("Cart_Total_Products"),
    col("cart_total_quantity").cast("int").alias("Cart_Total_Quantity"),
    col("ingestion_time")
)



StatementMeta(, 9f6bb14d-2ee8-464f-89c0-fadc41cfda23, 5, Finished, Available, Finished)

In [5]:
# 3️⃣ Drop duplicates
resultant_df = resultant_df.dropDuplicates(["Cart_ID", "Product_ID"])

# Add Order_Total column
resultant_df = resultant_df.withColumn("Cart_Line_Total", (round(col("Price") * col("Quantity"), 2)))

# 4️⃣ Add silver ingestion time
resultant_df = resultant_df.withColumn("silver_ingestion_time", current_timestamp())

# 5️⃣ Write to Silver
resultant_df.write.format("delta").mode("overwrite").saveAsTable("Slv_Carts")

StatementMeta(, 9f6bb14d-2ee8-464f-89c0-fadc41cfda23, 7, Finished, Available, Finished)